In [ ]:
!pip install datasets evaluate tqdm -q
!pip install -U accelerate --quiet

In [ ]:
!pip install -U -q PyDrive


In [ ]:
!pip install gdown


In [ ]:
import gdown

# Replace 'your_file_id' with the actual file ID from Google Drive
file_id = '16KcQSmdQ7dtQHgcQkP8I4IvSsd9lNqYK'
output_path = '/kaggle/working/mbart.zip'

gdown.download(f'https://drive.google.com/uc?id={file_id}', output_path, quiet=False)




In [ ]:
# Unzip the model
!unzip /kaggle/working/mbart.zip -d /kaggle/working/mbart/

In [ ]:
import pandas as pd
import numpy as np
from transformers import ( AutoModelForSeq2SeqLM, AutoTokenizer,MBartForConditionalGeneration,
                         pipeline)
import torch
import os

In [ ]:
def extract_summary(generated_text):
    prefix_to_remove = "Summary :"
    prefix_to_remove2 = "Summary:"
    generated_text=str(generated_text)
    if generated_text.startswith(prefix_to_remove):
        cleaned_text = generated_text[len(prefix_to_remove):]
    else:
        cleaned_text = generated_text
    if generated_text.startswith(prefix_to_remove2):
        cleaned_text2 = cleaned_text[len(prefix_to_remove2):]
    else:
        cleaned_text2 = cleaned_text
    return cleaned_text2

In [ ]:
def process_news_prompt(prompt, max_tensor_size=1024):
    inputs = tokenizer(prompt, return_tensors="tf")

    # Get the size of the input tensor
    tensor_size = inputs["input_ids"].shape[1]
    print(tensor_size)
    # Check if the tensor size is greater than 1024
    if tensor_size > max_tensor_size:
        # Shorten the prompt to fit within 1024 tokens
        num_tokens_to_remove = tensor_size - max_tensor_size
        prompt = tokenizer.decode(inputs["input_ids"][0, :-num_tokens_to_remove])
    prefix_to_remove = "en_XX "
    if prompt.startswith(prefix_to_remove):
        cleaned_text = prompt[len(prefix_to_remove):]
    else:
        cleaned_text = prompt
    # Return the reduced prompt as text
    return cleaned_text

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,MBartForConditionalGeneration


In [ ]:
model_ckpt = "/kaggle/working/mbart/finetunedmBartLarge67kWith50news147Scripts"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = MBartForConditionalGeneration.from_pretrained(model_ckpt)

In [ ]:
prompt = """Text :اردو زبان کو اُردو تختی اور جموں کشمیر کی ادبی زبان قرار دیا جاتا ہے۔ یہ زبان مشرقی پاکستان میں بولی جاتی ہے اور اس کے علاوہ ہندوستان کے کئی حصوں میں بھی استعمال ہوتی ہے۔ اردو کا تعلق عربی اور فارسی زبانوں سے ہے، اور اس کا لکھائی ادبی عربی سے مشابہت رکھتا ہے۔ اس کی لکھائی نستعلیق، نستعلیق نگار، اور نستعلیقِ نگار نستعلیق کہلاتی ہیں۔

اردو زبان کی تاریخ بہت پرانی ہے اور اس کی شروعات قدیم ہندوستانی زبانوں سے ملتی ہے۔ اردو کی اختصاری شکل ہے 'اردو' جو 'اردو زبان کا مخفف ہے۔ اردو میں نثر اور نظم کا بہترین اظہار کیا جاتا ہے، اور اس زبان کے شاعر وادیئے، میرزا غالب، اور فیض احمد فیض جیسے معروف ادیب بڑے مقام حاصل کر چکے ہیں۔

اس دورانِ حاضر، ٹیکنالوجی کے فیصلے کرنے والے لوگ بھی اردو کے استعمال کے فوائد سمجھ رہے ہیں۔ اردو میں ٹیکسٹ سمارٹ فون، کمپیوٹر، اور انٹرنیٹ کی مختلف اپلیکیشنز کے لئے استعمال ہوتا ہے۔ اردو کے لئے بھی مخصوص نئے الفاظ اور ٹیکنالوجی کے مصطلحات تیار کیے جا رہے ہیں تاکہ زبان کی ترویج اور ترقی میں مدد فراہم کی جائے۔"""

model


In [ ]:
prompt = """وسائیڈل :م: کی لسٹ میں آتے ہوں-ٹھیک ہے مطلب سوسائیڈ سے ریلیٹڈ ہوں جیسے خود کشی میں نے بتایا یا پھر کوئی ہارم بتا دیا اپنے آپ کو نقصان پہنچانا-جی-سیلف-سیلف انجری ہو گیا-جی-سیلف ڈسٹرکشن ہو گئی-اس طرح-جی-اس طرح کے کی ورڈز جی جی بالکل ایسا ہی ہے-(())-ٹھیک ہے سر-یہ بھی دھیان رکھنا ہے آپ لوگوں نے کہ اس کی لسٹ جو ہے ناں وہ لمیٹڈ نہ ہو کافی ویریئیشن آئے ڈائیورسٹی ہے تاکہ ہمارے پاس ایک :م: کافی ذیادہ ناں :م: وہ لسٹ بن جائے ورڈز کی-اور ہمارے لیے-((ایزی))-ہو جائے-((گا))-جی سر :م: یعنی اس میں بائسنیس نہ آئے پھر-جی جی بالکل بائسنیس نہ-<background> </background> بائسنیس-تو ہمارے لیے کافی زیادہ کام خراب ہو گا-:م: مطلب یہ ہے کہ آپ لوگ جو ڈیٹا سیٹ بنا رہے ہیں ناں وہ بالکل ایکوریٹ اور بالکل :م: سمجھ لیں اچھا بنانا ٹھیک بنانا-:م: اوکے سر ہم پوری کوشش کریں گے اپنی-ہاں جی اس خیال کا :م: یہ بھی دھیان رکھنا ہے کہ ماڈل جو ہے وہ ایکوریٹلی لرن کرے-ٹھیک ہے سر-<cough/>-:م: ٹھیک-((ہے))-اوکے :م: ہاں ہاں جب آپ کو پروفیشنلز کی :م: ضرورت پڑے آپ لوگوں نے مجھے بتانا ہے تاکہ ہم ناں ان سے تھوڑی سی گائیڈنس لے سکے اگر ہمارے سے کوئی مسٹیک ہو جائے-ج"""

In [ ]:
# prompt=dataset_test['Question'][9]
prompt='Text :'+prompt
prompt

In [ ]:
size=tokenizer(prompt, return_tensors="tf")["input_ids"].shape[1]
size

In [ ]:
from transformers import pipeline


In [ ]:
import gc
gc.collect()

In [ ]:
generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer, do_sample=False)
result = generator(prompt, max_length=size+100)

In [ ]:
result

In [ ]:
import os
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load your custom model and tokenizer
model_path = "/kaggle/working/mbart/finetunedmBartLarge67kWith50news147Scripts"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Create a directory to store the modified CSV files
output_directory = "/kaggle/working/output"
os.makedirs(output_directory, exist_ok=True)

# Initialize the text generation pipeline
generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

# Function to process each CSV file
def process_csv(input_path, output_directory):
    # Read the CSV file
    df = pd.read_csv(input_path)
    
    # Generate output based on the first column's content for each row
    for index, row in df.iterrows():
        input_text = row['reference']
        output_text = generator(input_text)[0]['generated_text']
        
        # Update the 'Output' column with the generated text
        df.at[index, 'Output'] = output_text
        
        # Print a message for each row processed
        print(f"Processed row {index + 1} in file: {input_path}")
    
    # Create the output path for the modified CSV file
    output_path = os.path.join(output_directory, os.path.basename(input_path))
    
    # Save the modified CSV file
    df.to_csv(output_path, index=False)
    
    # Print a message when processing is complete for the file
    print(f"Processing complete for file: {input_path}")

# Directory containing the CSV files
input_directory = "/kaggle/input/my-test/Test_files"

# Iterate over each CSV file in the directory
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        input_path = os.path.join(input_directory, filename)
        process_csv(input_path, output_directory)


In [ ]:
import os
import pandas as pd

# Function to remove prefixes from the second column
def remove_prefix(text, prefix_list):
    for prefix in prefix_list:
        if text.startswith(prefix):
            return text[len(prefix):].strip()
    return text

# Function to process each CSV file and remove prefixes from the second column
def process_csv(input_path, output_directory, prefix_list):
    # Read the CSV file
    df = pd.read_csv(input_path)
    
    # Remove prefixes from the second column
    df['Output'] = df['Output'].apply(lambda x: remove_prefix(x, prefix_list))
    
    # Create the output path for the modified CSV file
    output_path = os.path.join(output_directory, os.path.basename(input_path))
    
    # Save the modified CSV file
    df.to_csv(output_path, index=False)
    
    # Print a message when processing is complete for the file
    print(f"Processing complete for file: {input_path}")

# Directory containing the CSV files
input_directory = "/kaggle/working/output"
output_directory = "/kaggle/working/removed_prefix_output2"
os.makedirs(output_directory, exist_ok=True)

# List of prefixes to remove
prefix_list = ["Summary :", "Summary ", "Summary Summary : ", "Summary:", "Summary Summary:"]

# Iterate over each CSV file in the directory
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        input_path = os.path.join(input_directory, filename)
        process_csv(input_path, output_directory, prefix_list)


In [ ]:
import pandas as pd

# Specify the path to the CSV file
csv_file_path = "/kaggle/working/removed_prefix_output/MR240-test.csv"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Display the DataFrame
print(df)


In [ ]:
import os
import zipfile
from IPython.display import FileLink

def zip_directory(directory, zip_name):
    # Create a zip file containing all files and subdirectories within the directory
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(directory):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(directory, '..')))

# Directory to be zipped and downloaded
directory_to_download = "/kaggle/working/removed_prefix_output"

# Name for the zip file
zip_file_name = "downloaded_directory.zip"

# Zip the directory
zip_directory(directory_to_download, zip_file_name)

# Provide the zip file for download
FileLink(zip_file_name)


In [ ]:
import os

# Directory containing the CSV files
input_directory = "/kaggle/working/removed_prefix_output"

# Iterate over each CSV file in the directory
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        input_path = os.path.join(input_directory, filename)
        
        # Read the CSV file and extract the data from the second column
        with open(input_path, 'r') as file:
            lines = file.readlines()
            second_column = [line.split(',')[1].strip() for line in lines[1:]]  # Assuming CSV format
            
            # Concatenate the data from the second column into a single string
            concatenated_data = '\n'.join(second_column)
            
            # Write the concatenated data to a text file
            output_file_path = os.path.splitext(input_path)[0] + "_concatenated.txt"
            with open(output_file_path, 'w') as output_file:
                output_file.write(concatenated_data)


In [ ]:
# Specify the path to the text file
txt_file_path = "/kaggle/working/removed_prefix_output/MR241-test_concatenated.txt"

# Read the contents of the text file
with open(txt_file_path, 'r') as file:
    file_contents = file.read()

# Display the contents of the text file
print(file_contents)


In [ ]:
import os

# Directory containing the CSV files
input_directory = "/kaggle/working/removed_prefix_output2"

# Directory to store the output text files
output_directory = "/kaggle/working/concatenated_files2"
os.makedirs(output_directory, exist_ok=True)

# Iterate over each CSV file in the directory
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        input_path = os.path.join(input_directory, filename)
        
        # Read the CSV file and extract the data from the second column
        with open(input_path, 'r') as file:
            lines = file.readlines()
            second_column = [line.split(',')[1].strip() for line in lines[1:]]  # Assuming CSV format
            
            # Concatenate the data from the second column into a single string
            concatenated_data = '\n'.join(second_column)
            
            # Specify the output file path in the output directory
            output_file_path = os.path.join(output_directory, os.path.splitext(filename)[0] + "_concatenated.txt")
            
            # Write the concatenated data to a text file in the output directory
            with open(output_file_path, 'w') as output_file:
                output_file.write(concatenated_data)


now remove the lines among the lines of each file and output in a separate directory

In [ ]:
import os

# Function to remove spaces and newlines between lines
def join_lines(lines):
    return ' '.join(line.strip() for line in lines)

# Directory containing the text files
input_directory = "/kaggle/working/concatenated_files2"

# Directory to store the output text files
output_directory = "/kaggle/working/concatenated_paragraphs2"
os.makedirs(output_directory, exist_ok=True)

# Iterate over each text file in the directory
for filename in os.listdir(input_directory):
    if filename.endswith(".txt"):
        input_path = os.path.join(input_directory, filename)
        
        # Read the text file and join lines into a single paragraph
        with open(input_path, 'r') as file:
            lines = file.readlines()
            paragraph = join_lines(lines)
            
            # Specify the output file path in the output directory
            output_file_path = os.path.join(output_directory, os.path.splitext(filename)[0] + "_paragraph.txt")
            
            # Write the paragraph to a text file in the output directory
            with open(output_file_path, 'w') as output_file:
                output_file.write(paragraph)


In [ ]:
# Specify the path to the text file
txt_file_path = "/kaggle/working/concatenated_paragraphs2/MR053_test_concatenated_paragraph.txt"

# Read the contents of the text file
with open(txt_file_path, 'r') as file:
    file_contents = file.read()

# Display the contents of the text file
print(file_contents)


In [ ]:
import os
import zipfile
from IPython.display import FileLink

def zip_directory(directory, zip_name):
    # Create a zip file containing all files and subdirectories within the directory
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(directory):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(directory, '..')))

# Directory to be zipped and downloaded
directory_to_download = "/kaggle/working/concatenated_paragraphs2"

# Name for the zip file
zip_file_name = "mBart-summaries.zip"

# Zip the directory
zip_directory(directory_to_download, zip_file_name)

# Provide the zip file for download
FileLink(zip_file_name)


In [ ]:
!pip install rouge-score


In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# Load your custom model and tokenizer
model_path = "/kaggle/working/mbart/finetunedmBartLarge67kWith50news147Scripts"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize the text generation pipeline
generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer, do_sample=False)

# Function to tokenize text
def tokenize_text(text):
    return tokenizer(text, return_tensors="pt")["input_ids"]

# Function to calculate BLEU score
def calculate_bleu(reference, generated):
    reference_tokens = tokenize_text(reference)
    generated_tokens = tokenize_text(generated)
    return sentence_bleu([reference_tokens], generated_tokens)

# Function to calculate ROUGE scores
def calculate_rouge(reference, generated):
    scores = scorer.score(reference, generated)
    return scores['rouge1'].fmeasure, scores['rouge2'].fmeasure, scores['rougeL'].fmeasure

# Load the CSV file
csv_file_path = "/kaggle/input/mbart-comparison-csv/mBart-comparison.csv"
df = pd.read_csv(csv_file_path)

# Apply the functions to each row and create new columns
df['BLEU'] = df.apply(lambda row: calculate_bleu(row['reference'], row['generated']), axis=1)
df['Rouge-1'], df['Rouge-2'], df['Rouge-L'] = zip(*df.apply(lambda row: calculate_rouge(row['reference'], row['generated']), axis=1))

# Save the DataFrame to a new CSV file
output_csv_file_path = "/kaggle/working/all.csv"
df.to_csv(output_csv_file_path, index=False)


In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load your custom model and tokenizer
model_path = "/kaggle/working/mbart/finetunedmBartLarge67kWith50news147Scripts"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to tokenize text
def tokenize_text(texts):
    return [tokenizer(sent, return_tensors="pt")["input_ids"] for sent in texts]

# Function to calculate BLEU score
def calculate_bleu(reference, generated):
    reference_tokens = tokenize_text(reference)
    generated_tokens = tokenize_text(generated)
    return sentence_bleu(reference_tokens, generated_tokens)

# Function to calculate ROUGE scores
def calculate_rouge(reference, generated):
    scores = scorer.score(reference, generated)
    return scores['rouge1'].fmeasure, scores['rouge2'].fmeasure, scores['rougeL'].fmeasure

# Load the CSV file
csv_file_path = "/kaggle/input/mbart-comparison-csv/mBart-comparison.csv"
df = pd.read_csv(csv_file_path)

# Apply the functions to each row and create new columns
df['BLEU'] = df.apply(lambda row: calculate_bleu([row['reference']], [row['generated']]), axis=1)
df['Rouge-1'], df['Rouge-2'], df['Rouge-L'] = zip(*df.apply(lambda row: calculate_rouge(row['reference'], row['generated']), axis=1))

# Save the DataFrame to a new CSV file
output_csv_file_path = "/kaggle/working/all2.csv"
df.to_csv(output_csv_file_path, index=False)
